Installations of needed dependencies


In [1]:
# Installations
!pip  install torch torchvision albumentations
!pip install -q kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/settings Access here and click " Create New TOken"  and upload here in collab the kaggle.json  file that makes you download

In [2]:
#Importing a Kaggle token (Kaggle.json)
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"hadjermohabeddine","key":"34cd6ea224fb3cd77a3aae4976d05ba9"}'}

In [3]:
# creating a kaggle hidden directory 
!mkdir ~/.kaggle
# moving the json imported file to the kaggle directory
!cp kaggle.json ~/.kaggle/
# giving the permissions to the json file
!chmod 600 ~/.kaggle/kaggle.json
# list all of the public datasets available on Kaggle
!kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                 title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
arnabchaki/data-science-salaries-2023               Data Science Salaries 2023  💸                25KB  2023-04-13 09:55:16           6978        173  1.0              
salvatorerastelli/spotify-and-youtube               Spotify and Youtube                           9MB  2023-03-20 15:43:25          12098        415  1.0              
iammustafatz/diabetes-prediction-dataset            Diabetes prediction dataset                 734KB  2023-04-08 06:11:45           3659         59  1.0              
shawkyelgendy/furniture-price-prediction            Furniture Price Prediction                   81K

Downloading the database imagenetmini

In [ ]:
#download
!kaggle datasets download -d ifigotin/imagenetmini-1000
#unzipping the dataset
!unzip /content/imagenetmini-1000.zip

imagenetmini-1000.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/imagenetmini-1000.zip
replace imagenet-mini/train/n01440764/n01440764_10043.JPEG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Get pretrained ResNet50 model


In [ ]:
import torchvision.models as models

# getting the pretrained resnet50
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval()

In [ ]:
from PIL import Image
import numpy as np 
import pandas as pd 
import torch
import torchvision as tv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optims
import torchvision.transforms as transforms
from PIL import Image
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as TF

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

transform = TF.Compose([TF.ToTensor(), 
                        #changing randomly the brightness, the contrast and the hue
                        transforms.ColorJitter(brightness=0.8, contrast=0.8, hue=0.5),
                        #rendomly flipping horizontally and vertically transformations
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomVerticalFlip(),
                        # random rotation between -90 and 90 degrees
                        transforms.RandomRotation((-90, 90)),
                        # bluriness transformation
                        transforms.GaussianBlur(21,0.1)])

# function to resize image to the size (128, 128) by default
def resize_image(src, size=(128, 128), bgc="white"):
    src.thumbnail(size, Image.ANTIALIAS)
    new_image = Image.new("RGB", size, bgc)    
    new_image.paste(src, (int((size[0]-src.size[0]) / 2)), int((size[1] - src.size[1]) / 2))
    return new_image


def load(path):
    # loading the dataset with the given transformation
    dataset = tv.datasets.ImageFolder(root=path, transform=transform)#,transform = Transforms())
    # deviding the size of the dataset (70% for the training and 30% for the test)
    train_size = int(0.7 * len(dataset))
    test_size = len(dataset) - train_size
    # getting the training and the test dataset 
    train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
    
    # creating the training dataloader
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )
    
    # creating the test dataloader
    test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=1,
        num_workers=0,
        shuffle=False
    )

    return train_loader, test_loader


In [ ]:
# loading the training data
train_loader, test_loader = load('/content/imagenet-mini/train')
#print(f"train loaders: {train_loader}")

In [ ]:
#default classes of resnet50
classes = models.ResNet50_Weights.DEFAULT.meta["categories"]
#print("Classes : ", classes)

Inferencing ResNet50

In [ ]:
import matplotlib.pyplot as plt
 

device = "cuda:0"
model.to(device)

# initiating the index to limit the traited images to 30
idx = 0

# initiating the original classes and predicted classes lists 
original_classes = []
predicted_classes = []

# inferencing the model
for epoch, (inputs, labels) in enumerate(train_loader):
    #print(f'Epoch {epoch+1}')  
          
    # Move input and label tensors to the device
    inputs = inputs.to(device)
    #print("Orig : ", labels)
    labels = classes[labels.to(device)]
    #print(inputs.shape)
    # the result of the image inference
    outputs = model(inputs.type( torch.cuda.FloatTensor ))
    #print("outputs shape : ", outputs.shape)
    outputs = outputs[0].argmax() 
    print("outputs  : ", outputs)
    # ploting the figures with thers original labels and predected labels
    plt.figure()
    plt.title(f"Original Class : {labels}, Predicted Class : {classes[outputs]}")
    img = np.transpose(inputs.detach().cpu().numpy()[0], (1, 2, 0))
    plt.imshow(img)
    plt.show()
    
    original_classes.append(labels)
    predicted_classes.append(classes[outputs])
    # giving a condition about the image samples that we want
    if idx == 30: break
    idx += 1

Metrics measures

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score

# Printing metrics results
print("PRECISION SCORE : ", precision_score(original_classes, predicted_classes, average='macro'))
print("RECALL SCORE : ", recall_score(original_classes, predicted_classes, average='macro'))
print("ACCURACY SCORE : ", accuracy_score(original_classes, predicted_classes))
print("F1_SCORE SCORE : ", f1_score(original_classes, predicted_classes, average='macro'))